In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import  Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.common.by import  By 
from selenium.webdriver.support import  expected_conditions as EC 

import pandas as pd
import numpy as np
import time
import datetime

In [2]:
#建立数据库
columns=['时间','发布日期','作者名','作者类型','粉丝数','投稿数','视频名','分区','子分区','视频类型','自制or转载','标签','视频长度','播放量','点赞数','投币数','收藏数','转发数','评论数','弹幕数','有无分集']
channel_shishang_jianshen=pd.DataFrame(columns=columns)
channel_shishang_jianshen

,时间,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,...,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集


In [22]:
driver=webdriver.Chrome()
driver.get('https://passport.bilibili.com/login') 

In [23]:
#登录
driver.find_element_by_xpath('//*[@id="login-username"]').send_keys('15717506191')
driver.find_element_by_xpath('//*[@id="login-passwd"]').send_keys('xy000000')
driver.find_element_by_xpath('//*[@id="geetest-wrap"]/div/div[5]/a[1]').click()
#需要手动过验证码

In [24]:
driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[11]/div/a/span').click()  #选择“时尚”频道

In [25]:
#获得分区名
channel=driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[11]/div/a/span').text
channel=channel.strip('0123456789+')
channel

'时尚'

In [26]:
driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[4]/a').click()  #选择“健身”子频道
sub_channel=driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[4]/a').text
sub_channel

'健身'

In [27]:
driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[1]/ul/a[2]/li').click() #让视频按热度排序

In [32]:
print(datetime.datetime.now())

for month in range(7,11): 
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div/span').click()
    period=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).text
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).click()
    WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]/div/div[2]/div[3]/a'))) 
    for page in range(5):    #爬取每个频道1月到10月每个月播放量前100的视频

        video_loc=1
        while video_loc<21:

            ### 判断作者类型###
            
            
            driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).click()
            driver.switch_to_window(driver.window_handles[-1])
            
            
            author_web_show='No'
            try:
                i=0
                while author_web_show=='No':
                    driver.refresh()
                    i+=1
                    try:
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="n-fs"]')))  #看作者名是否出现
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="navigator"]/div/div[1]/div[1]/a[3]/span[3]')))  #看投稿数是否出现
                        author_web_show='Yes'
                    except:
                        author_web_show='No' 
                        if i>3:  
                            raise Exception  #刷新几次后仍爬不了说明用户已注销

                try:
                    driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m organization-auth"]')
                    author_type='机构认证'

                except:
                    try:
                        driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m personal-auth"]')
                        author_type='个人认证'
                    except:
                        author_type='无'

                fans_num=driver.find_element_by_xpath('//*[@id="n-fs"]').text  #获取作者粉丝数

                ### 获取作者投稿数 ###
                driver.find_element_by_xpath('//*[@class="n-num"]').click()
                WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span')))  
                video_num=driver.find_element_by_xpath('//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span').text

                driver.close()
                driver.switch_to_window(driver.window_handles[-1])


                view_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[1]/span'%video_loc).text #获取播放量
                real_dan_mu_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[2]/span'%video_loc).text #获取弹幕数
                author=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).text #获取作者名
                driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/a'%video_loc).click() #进入视频界面  
                driver.switch_to_window(driver.window_handles[-1]) 

                try:
                    WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))  #若出现视频被删除的情况则退出

                    video_web_show='No'
                    while video_web_show=='No':
                        driver.refresh()

                        try:
                            WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@class="bilibili-player-video-time-total"]'))) #以视频长度出现时间判定网页是否加载完
                            video_web_show='Yes'

                            try:      
                                driver.find_element_by_xpath('//*[@id="multi_page"]')   #测试是否有分集视频
                                multi='是'

                            except:
                                multi='否'

                            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
                            driver.execute_script('window.scrollTo(document.body.scrollHeight,0)') #再调回最上面是因为，视频长度的元素必须在屏幕内才可以被提取，否则为空

                            bv=driver.current_url[33:44] # 获取BV号（BV号在网址中）
                            print('开始BV号为%s视频属性的抓取'%bv)
                            video_name=driver.find_element_by_xpath('//*[@id="viewbox_report"]/h1/span').text #获取视频名
                            
                            ### 获取发布日期 ###
                            release_date=driver.find_element_by_xpath('//*[@id="viewbox_report"]/div/span[3]').text
                            
                            ### 获取视频类型 ###
                            try:
                                video_type=driver.find_element_by_xpath('//*[@class="activity"]').text
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type=video_type+'和合作视频'
                                except:
                                    pass
                            except:
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type='合作视频'
                                except:
                                    video_type='普通视频'


                            ### 获取点赞数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text=='点赞':
                                like_num=0
                            else:
                                like_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text 

                            ### 获取投币数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text=='投币':
                                coin_num=0
                            else:
                                coin_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text

                            ### 获取收藏数 ###             
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text=='收藏':
                                collect_num=0
                            else:
                                collect_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text

                            ### 获取转发数 ###       
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text=='分享':
                                forward_num=0
                            else:
                                forward_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text

                            ### 获取视频长度 ###

                            video_length=driver.find_element_by_xpath('//*[@class="bilibili-player-video-time-total"]').text  

                            ### 判断自制还是转载 ###
                            driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').click()
                            try:
                                driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/div[2]/div[2]').text
                                self_made='自制'
                            except:
                                self_made='转载'
                            driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/i').click()

                            ### 获取标签 ###
                            k=1
                            labels=np.array
                            while True:
                                try:
                                    labels=np.append(labels,driver.find_element_by_xpath('//*[@id="v_tag"]/ul/li[%d]'%k).text)
                                    k+=1
                                except: 
                                    break
                            labels=labels[1:]

                            ### 获取评论数 ###
                            driver.execute_script('window.scrollTo(0, 700)')        
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[2]'))) #保证"评论"二字能加载出来                                                   
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div')))
                            if driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/div/div[4]/div').text=='没有更多信息':    #应对评论区被关闭的情况
                                comment_num='NaN'
                            else:
                                while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                    try:
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div[1]')))
                                        while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                            time.sleep(0.1) #很奇怪，评论数可以定位的一瞬间并不能爬取，会返回空
                                        comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text     
                                    except:
                                        driver.refresh()    #有的时候“评论”二字出现，评论数不出现，页面好像卡住了
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))
                                        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[1]')))

                                comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text

                            channel_shishang_jianshen.loc[bv]=[period,release_date,author,author_type,fans_num,video_num,video_name,channel,sub_channel,video_type,self_made,labels,video_length,view_num,like_num,coin_num,collect_num,forward_num,comment_num,real_dan_mu_num,multi]

                            print('BV号为%s的视频属性抓取结束'%bv)

                            driver.close()
                            driver.switch_to_window(driver.window_handles[-1])

                        except:
                            video_web_show='No' 
                except:
                    driver.close()
                    driver.switch_to_window(driver.window_handles[-1])
            except:
                driver.close()
                driver.switch_to_window(driver.window_handles[-1])
            
            video_loc+=1


        driver.find_element_by_xpath('//*[@class="nav-btn iconfont icon-arrowdown3"]').click() #点击下一页
        WebDriverWait(driver,100).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]')))  #点击下一页后等页面刷新
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[2]').click()  #回到第1页
    WebDriverWait(driver,100).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]')))
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]').click()  #发现很多月份漏爬了约20个，而且恰好是第一页，所以再点一次第一页
        
        
print(datetime.datetime.now()) 

2020-12-23 09:25:14.683077


<ipython-input-32-c239f23e0a4a>:17: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-32-c239f23e0a4a>:54: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-32-c239f23e0a4a>:61: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1vi4y137Ab视频属性的抓取
BV号为1vi4y137Ab的视频属性抓取结束


<ipython-input-32-c239f23e0a4a>:181: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1sz411v7JJ视频属性的抓取
BV号为1sz411v7JJ的视频属性抓取结束
开始BV号为1mV41167i5视频属性的抓取
BV号为1mV41167i5的视频属性抓取结束
开始BV号为1tV41167YG视频属性的抓取
BV号为1tV41167YG的视频属性抓取结束
开始BV号为1BC4y187NG视频属性的抓取
BV号为1BC4y187NG的视频属性抓取结束
开始BV号为1hZ4y1T7cp视频属性的抓取
BV号为1hZ4y1T7cp的视频属性抓取结束
开始BV号为1F54y1S7dt视频属性的抓取
BV号为1F54y1S7dt的视频属性抓取结束
开始BV号为1ak4y1m7CV视频属性的抓取
开始BV号为1ak4y1m7CV视频属性的抓取
BV号为1ak4y1m7CV的视频属性抓取结束
开始BV号为1Ff4y1R7CV视频属性的抓取
BV号为1Ff4y1R7CV的视频属性抓取结束
开始BV号为1RT4y177ni视频属性的抓取
BV号为1RT4y177ni的视频属性抓取结束
开始BV号为1zA411e71M视频属性的抓取
BV号为1zA411e71M的视频属性抓取结束


<ipython-input-32-c239f23e0a4a>:190: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为18V411z7Xk视频属性的抓取
BV号为18V411z7Xk的视频属性抓取结束
开始BV号为18D4y1Q7RD视频属性的抓取
BV号为18D4y1Q7RD的视频属性抓取结束
开始BV号为1sA411i78E视频属性的抓取
BV号为1sA411i78E的视频属性抓取结束
开始BV号为1Lg4y1i7t4视频属性的抓取
BV号为1Lg4y1i7t4的视频属性抓取结束
开始BV号为1wZ4y1u78o视频属性的抓取
BV号为1wZ4y1u78o的视频属性抓取结束
开始BV号为1Vz4y1D7Nd视频属性的抓取
BV号为1Vz4y1D7Nd的视频属性抓取结束
开始BV号为1R5411h7Wo视频属性的抓取
BV号为1R5411h7Wo的视频属性抓取结束
开始BV号为1zt4y1Q7Jh视频属性的抓取
BV号为1zt4y1Q7Jh的视频属性抓取结束
开始BV号为1M54y1q7z1视频属性的抓取
BV号为1M54y1q7z1的视频属性抓取结束
开始BV号为16a4y1a7rb视频属性的抓取
BV号为16a4y1a7rb的视频属性抓取结束
开始BV号为1da4y1Y7A4视频属性的抓取
BV号为1da4y1Y7A4的视频属性抓取结束
开始BV号为1EK411J7nd视频属性的抓取
BV号为1EK411J7nd的视频属性抓取结束
开始BV号为1Nz4y1X7SB视频属性的抓取
BV号为1Nz4y1X7SB的视频属性抓取结束
开始BV号为1Vh411Z7Kk视频属性的抓取
BV号为1Vh411Z7Kk的视频属性抓取结束
开始BV号为13T4y1E7S8视频属性的抓取
BV号为13T4y1E7S8的视频属性抓取结束
开始BV号为1ch411Z7tj视频属性的抓取
BV号为1ch411Z7tj的视频属性抓取结束
开始BV号为1a54y1q7vu视频属性的抓取
BV号为1a54y1q7vu的视频属性抓取结束
开始BV号为13A411Y78i视频属性的抓取
BV号为13A411Y78i的视频属性抓取结束
开始BV号为1FZ4y1M7ag视频属性的抓取
BV号为1FZ4y1M7ag的视频属性抓取结束
开始BV号为1sz4y1D7Ya视频属性的抓取
BV号为1sz4y1D7Ya的视频属性抓取结束
开始BV号为1VK4y1x7HQ视频属性的抓取
BV号为1VK4y1x7HQ的视

<ipython-input-32-c239f23e0a4a>:187: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1Ep4y1i7W8视频属性的抓取
BV号为1Ep4y1i7W8的视频属性抓取结束
开始BV号为1mV411279t视频属性的抓取
BV号为1mV411279t的视频属性抓取结束
开始BV号为1oh411R7Rr视频属性的抓取
BV号为1oh411R7Rr的视频属性抓取结束
开始BV号为1pK411T77p视频属性的抓取
BV号为1pK411T77p的视频属性抓取结束
开始BV号为1BA411n7aw视频属性的抓取
BV号为1BA411n7aw的视频属性抓取结束
开始BV号为1b54y1i7qk视频属性的抓取
BV号为1b54y1i7qk的视频属性抓取结束
开始BV号为1BK4y1Y7Gz视频属性的抓取
BV号为1BK4y1Y7Gz的视频属性抓取结束
开始BV号为1J5411b7T1视频属性的抓取
BV号为1J5411b7T1的视频属性抓取结束
开始BV号为1s64y1c78k视频属性的抓取
BV号为1s64y1c78k的视频属性抓取结束
开始BV号为1Mh411o79i视频属性的抓取
BV号为1Mh411o79i的视频属性抓取结束
开始BV号为1Q64y1F75H视频属性的抓取
BV号为1Q64y1F75H的视频属性抓取结束
开始BV号为1WD4y1m7MV视频属性的抓取
BV号为1WD4y1m7MV的视频属性抓取结束
开始BV号为1KA411e7zp视频属性的抓取
BV号为1KA411e7zp的视频属性抓取结束
开始BV号为1KT4y177Dm视频属性的抓取
开始BV号为1KT4y177Dm视频属性的抓取
BV号为1KT4y177Dm的视频属性抓取结束
开始BV号为1QD4y1d7xP视频属性的抓取
BV号为1QD4y1d7xP的视频属性抓取结束
开始BV号为1664y1c7Gy视频属性的抓取
BV号为1664y1c7Gy的视频属性抓取结束
开始BV号为1fV411z7Jj视频属性的抓取
BV号为1fV411z7Jj的视频属性抓取结束
开始BV号为1EK4y1Y7cW视频属性的抓取
BV号为1EK4y1Y7cW的视频属性抓取结束
开始BV号为1wV411m76g视频属性的抓取
BV号为1wV411m76g的视频属性抓取结束
开始BV号为1yK4y1v7fL视频属性的抓取
BV号为1yK4y1v7fL的视频属性抓取结束
开始BV号为1ei4y1g7tM

开始BV号为1hy4y1r7KV视频属性的抓取
BV号为1hy4y1r7KV的视频属性抓取结束
开始BV号为1hf4y1i7gC视频属性的抓取
BV号为1hf4y1i7gC的视频属性抓取结束
开始BV号为1Ya4y1L7u1视频属性的抓取
BV号为1Ya4y1L7u1的视频属性抓取结束
开始BV号为1bk4y1C7zE视频属性的抓取
BV号为1bk4y1C7zE的视频属性抓取结束
开始BV号为1Kr4y1w7gW视频属性的抓取
BV号为1Kr4y1w7gW的视频属性抓取结束
开始BV号为1CD4y197x2视频属性的抓取
BV号为1CD4y197x2的视频属性抓取结束
开始BV号为1wi4y177ww视频属性的抓取
BV号为1wi4y177ww的视频属性抓取结束
开始BV号为1N54y127Fw视频属性的抓取
BV号为1N54y127Fw的视频属性抓取结束
开始BV号为1c5411L7KV视频属性的抓取
BV号为1c5411L7KV的视频属性抓取结束
开始BV号为1bZ4y157eo视频属性的抓取
BV号为1bZ4y157eo的视频属性抓取结束
开始BV号为1ZK411A72t视频属性的抓取
BV号为1ZK411A72t的视频属性抓取结束
开始BV号为16t4y1q75e视频属性的抓取
BV号为16t4y1q75e的视频属性抓取结束
开始BV号为1az4y1o7CR视频属性的抓取
BV号为1az4y1o7CR的视频属性抓取结束
开始BV号为1GK4y1h7fV视频属性的抓取
BV号为1GK4y1h7fV的视频属性抓取结束
开始BV号为1ey4y1r7bu视频属性的抓取
BV号为1ey4y1r7bu的视频属性抓取结束
开始BV号为1954y167sD视频属性的抓取
BV号为1954y167sD的视频属性抓取结束
开始BV号为1854y1C7ab视频属性的抓取
BV号为1854y1C7ab的视频属性抓取结束
开始BV号为18v411k7JH视频属性的抓取
BV号为18v411k7JH的视频属性抓取结束
开始BV号为16V41117g6视频属性的抓取
BV号为16V41117g6的视频属性抓取结束
开始BV号为1hp4y1Y7Gs视频属性的抓取
BV号为1hp4y1Y7Gs的视频属性抓取结束
开始BV号为14D4y197bR视频属性的抓取
BV号为14D4y197bR的视

In [31]:
driver.switch_to_window(driver.window_handles[-1])

<ipython-input-31-dd0a1399d54d>:1: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


In [71]:
channel_shishang_jianshen.groupby('时间').agg('count')

,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,自制or转载,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
时间,,,,,,,,,,,,,,,,,,,,
10月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
1月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
2月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
3月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
4月,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98
5月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
6月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
7月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
8月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99


In [72]:
channel_shishang_jianshen.to_json('channel_shishang_jianshen.json')